# Tutorial 0: Before Starting
If you want to know about the relevant information and usage instructions about this library, please refer to the README.md or the README.en.md.

## Auxiliary parameter generation
Currently, we simulate the auxiliary parameters provided by a trusted third party using local files. If you want to generate these auxiliary parameters, please run```./debug/offline_parameter_generation.py```. The auxiliary parameters will be saved as a directory ```data``` by adding the base path .NssMPClib/data/ to the current user's home directory (Linux/Unix: /home/{username}; macOS: /Users/{username}; Windows: C:\Users\\{username}).

Additionally, you can change the number of parameters generated according to your needs. 
In this way, the parameters can be generated as follows: 

In [1]:
from NssMPC.crypto.aux_parameter import *
from NssMPC.crypto.aux_parameter.function_secret_sharing_keys.vsigma_key import VSigmaKey
from NssMPC.crypto.aux_parameter.select_keys.vos_key import VOSKey
from NssMPC.crypto.aux_parameter.truncation_keys.rss_trunc_aux_param import RssTruncAuxParams

gen_num = 100

AssMulTriples.gen_and_save(gen_num, saved_name='2PCBeaver', num_of_party=2, type_of_generation='TTP')
AssMulTriples.gen_and_save(gen_num, saved_name='3PCBeaver', num_of_party=3, type_of_generation='TTP')
BooleanTriples.gen_and_save(gen_num, num_of_party=2, type_of_generation='TTP')
Wrap.gen_and_save(gen_num)
GrottoDICFKey.gen_and_save(gen_num)
RssMulTriples.gen_and_save(gen_num)
DICFKey.gen_and_save(gen_num)
SigmaDICFKey.gen_and_save(gen_num)
ReciprocalSqrtKey.gen_and_save(gen_num)
DivKey.gen_and_save(gen_num)
GeLUKey.gen_and_save(gen_num)
RssTruncAuxParams.gen_and_save(gen_num)
B2AKey.gen_and_save(gen_num)
TanhKey.gen_and_save(gen_num)
MACKey.gen_and_save(gen_num)

VOSKey.gen_and_save(gen_num, 'VOSKey_0')
VOSKey.gen_and_save(gen_num, 'VOSKey_1')
VOSKey.gen_and_save(gen_num, 'VOSKey_2')

VSigmaKey.gen_and_save(gen_num, 'VSigmaKey_0')
VSigmaKey.gen_and_save(gen_num, 'VSigmaKey_1')
VSigmaKey.gen_and_save(gen_num, 'VSigmaKey_2')

B2AKey.gen_and_save(gen_num, 'B2AKey_0')
B2AKey.gen_and_save(gen_num, 'B2AKey_1')
B2AKey.gen_and_save(gen_num, 'B2AKey_2')


1. **Beaver Triples**
   Used for multiplication in various secret sharing schemes:
   * **AssMulTriples**: Multiplication triples for two-party and three-party Arithmetic Secret Sharing (ASS).
   * **BooleanTriples**: AND triples for two-party Boolean Secret Sharing.
   * **RssMulTriples**: Malicious-secure multiplication triples for three-party Replicated Secret Sharing (RSS).

2. **Auxiliary Parameters for Two-Party Semi-Honest Secure Comparison**
   * **DICFKey**
   * **GrottoDICFKey**
   * **SigmaDICFKey**

3. **Auxiliary Parameters for Complex Nonlinear Functions**
   Used to securely evaluate nonlinear functions via lookup tables or specialized preprocessing:
   * **GeLUKey**: Supports the Gaussian Error Linear Unit activation function.
   * **ReciprocalSqrtKey**: Supports reciprocal square root and negative exponent operations.
   * **DivKey**: Used for secure division.
   * **TanhKey**: Supports the tanh activation function.

4. **Auxiliary Parameters for Three-Party Malicious-Secure Protocols**
   Required for correctness verification and malicious security in three-party MPC protocols:
   * **MACKey**: Message Authentication Code key for malicious-secure verification.
   * **VOSKey**: Used for oblivious selection.
   * **VSigmaKey**: FSS key used in malicious-secure comparison via the verifiable sigma protocol.

5. **Other Functionality-Related Auxiliary Parameters**
   Supporting additional operations such as truncation and share-type conversion:
   * **Wrap**, **RssTruncAuxParams**: Used for truncation in different secret sharing schemes.
   * **B2AKey**: Converts Boolean Secret Sharing to Arithmetic Secret Sharing (Boolean-to-Arithmetic, B2A).


In addition to these keys, other parameters are also required for some operations, such as the matrix beaver triples for matrix multiplication, which is related to the size of the matrix involved in the operation. Such parameters are usually related to actual operations, so it is hard to generate them in advance, and the generation scheme will be shown in subsequent tutorials.

## Configuration file
Related configuration of the library will read the configuration file ```config.json```, which will be generated under the system path (Linux/Unix: /home/{username}; macOS: /Users/{username}; Windows: C:\Users\{username}). 
Configuration files are used to define the operating parameters of applications. These parameters can include database connection information, network settings, user permissions, and so on. Therefore, users can easily adjust the behavior of the application.
Now, let's have an insight into the basic configuration so that you can change the configuration to achieve different operations in the future. See the config section of the [API documentation](https://www.xidiannss.com/doc/NssMPClib/config.html) for more details.

## Some utils
In ```./NssMPC/common/utils/debug_utils.py```, we provide some tools to help you debug the code. For example, you can use the following code to check the time of a function:

In [ ]:
from NssMPC.common.utils import get_time
res = get_time(func, *args)

The utility function `get_time` is used to measure the execution time of any function. It returns the function’s output and prints the time consumed during execution. The parameters of `get_time` are the target function followed by the arguments to be passed to it.

In addition, we provide a function for evaluating the communication cost of secure multiparty computation tasks. You can measure the communication cost of a function using the following code:

In [ ]:
from NssMPC.common.utils import comm_count
res = comm_count(communicator, func, *args)

The `comm_count` function returns the result of the function and prints the total communication cost incurred during its execution. Its parameters include the Communicator instance from `NssMPC/common/network/communicator.py`, followed by the target function and its arguments.

You can also use the `statistic` function to evaluate both the runtime and communication cost of a given function. It performs multiple executions to provide stable measurements by averaging over several runs.

In [ ]:
from NssMPC.common.utils import statistic
res = statistic(func, *args, times=10, warmup=5)

The `statistic` function measures the average execution time and total communication cost. It first runs the function for a specified number of warm-up iterations (default: 5) to eliminate initialization overhead, and then executes it for a given number of measurement rounds (default: 10) to compute the mean runtime. Both times and warmup can be adjusted as needed.

#### Note
If you find that some functions mentioned in this tutorial can not run, don't worry. It may be because the auxiliary parameters required for some functions are not generated or the auxiliary parameters are insufficient. You can refer to the tutorial and codes in the```./debug``` package to generate the auxiliary parameters required according to your needs, and distribute the calculation to multiple parties.